In [4]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import hervpd as hp
import csvUtils as csvu
import parseActivityFiles as paf
import parseIntervalFiles as pif

from datetime import datetime, date, time, timedelta

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0 - Pipeline configuration 
* set the input/output directories, user id and verbose level

In [5]:
    verbose = True    
    user = 0
    
    RAW_DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/Raw/%d"%user
    PRE_DATA_PATH = "/home/ju/GDrive/Projects/HeRV/Data/PreProcessed/%d"%user
    
    # duration (in seconds) to be cropped from the beginning of each second to account for stabilization and user adjustment
    crop = 90
    
    # duration (in seconds) of each fragment to be sent to analysis
    duration = 300
    
    # if any fragment has more than 'threshold' consecutive seconds with no beats, it will be discarded
    threshold = 3

## 1 - Extract sessions from raw data (intervals + annotations files)

### 1.1 - List activities with session start/stop datetime
Parses activities files and prints all errors found (activities without start or stop, invalid dates, etc)

In [6]:
sessions = paf.parseActivityFiles(dirname=RAW_DATA_PATH)

reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171014.csv ... 
orphan stop in: ['2017-10-14 2:14:21', 'stop', '', '', '']
orphan start in: ['2017-10-14 11:58:31', 'start', 'focused-active', 'lying down', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171102.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171016.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171006.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171104.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171029.csv ... 
orphan stop in: ['2017-10-29 15:27:59', 'stop', '', '']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171028.csv ... 
orphan start in: ['2017-10-28 11:48:07', 'start', 'sleep', 'posture']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171013.csv ... 
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/act171003.csv ... 
orphan start in: ['2017-10-03 22:56:19', 'start', 'eat', 'sitting']
reading /home/ju/GDrive/Projects/HeRV/Data/Raw/0/

### 1.2 Extract session intervals

Retrieves from the heartbeat files all the intervals contained in each session's duration and adds them to the sessions objects (in memory)

In [7]:
for sess in sessions:            
    sess['rr'] = pif.getIntervals(sess['start'], sess['stop'], RAW_DATA_PATH)
    sess['countRR'] = len(sess['rr'])  
    

In [8]:
df = pd.DataFrame(sessions)
df.head()

,activity,countRR,duration,notes,posture,rr,start,stop
0,focused-active,2520,1969,,sitting,"[{'date': 2017-10-14 02:14:22, 'interval': 831...",2017-10-14 02:14:21,2017-10-14 02:47:10
1,focused-active,192,828,,sitting,"[{'date': 2017-10-14 02:57:33, 'interval': 819...",2017-10-14 02:57:32,2017-10-14 03:11:20
2,focused-active,1579,1236,,sitting,"[{'date': 2017-10-14 03:11:21, 'interval': 737...",2017-10-14 03:11:20,2017-10-14 03:31:56
3,focused-active,2064,1783,,sitting,"[{'date': 2017-10-14 03:31:58, 'interval': 920...",2017-10-14 03:31:57,2017-10-14 04:01:40
4,sleep,25409,23722,,lying down,"[{'date': 2017-10-14 04:05:26, 'interval': 739...",2017-10-14 04:05:25,2017-10-14 10:40:47


## 2 - Generate fragments from sessions